In [ ]:
import numpy as np
import sys; sys.path.append('..')
from bending_validation import suppress_stdout

import elastic_rods
from linkage_vis import LinkageViewer
from elastic_rods import EnergyType

In [ ]:
l = elastic_rods.RodLinkage('../../examples/sphere_open.obj', 10)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [0.008, 0.001])
l.setMaterial(mat)

In [ ]:
sphereView = LinkageViewer(l, width=768, height=512)
sphereView.setCameraParams(((1.0285379158600272, 3.657456693129147, 0.7152688786806418),
 (-0.05581693079870617, 0.15687135472757766, 0.9860404901940621),
 (-0.07426255785500183, -0.021637594449990902, -0.33950041086040444)))
sphereView.show()

In [ ]:
l.restLengthLaplacianEnergy(), l.energyBend(), l.energyStretch(), l.energyTwist(), l.energy()

In [ ]:
#with suppress_stdout(): elastic_rods.restlen_solve(l, niter=100, verbose=False)
sphereView.update()
driver=60
#driver=67
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
opts = elastic_rods.NewtonOptimizerOptions()
opts.niter = 1000
with suppress_stdout(): elastic_rods.compute_equilibrium(l, options=opts, fixedVars=fixedVars)
sphereView.update()

In [ ]:
l.averageJointAngle

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.useIdentityMetric = True
    opts.beta = 1e-8
#     opts.useNegativeCurvatureDirection = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
elastic_rods.benchmark_reset()
cr = open_linkage(l, driver, -1.25, 50, sphereView, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, verbose=False, useTargetAngleConstraint=True)
elastic_rods.benchmark_report()